In [2]:
import numpy as np
import tensorflow as tf
import cv2
import tensorflow_datasets as tfds

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [6]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape)
print(validation_targets.shape)

(6000, 28, 28, 1)
(6000,)


In [7]:
input_size = 784
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
NUM_EPOCHS = 30
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data,
          epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=2)

Epoch 1/30
540/540 - 7s - 13ms/step - accuracy: 0.8837 - loss: 0.4138 - val_accuracy: 0.9335 - val_loss: 0.2260
Epoch 2/30
540/540 - 7s - 13ms/step - accuracy: 0.9470 - loss: 0.1860 - val_accuracy: 0.9470 - val_loss: 0.1682
Epoch 3/30
540/540 - 3s - 6ms/step - accuracy: 0.9584 - loss: 0.1419 - val_accuracy: 0.9560 - val_loss: 0.1449
Epoch 4/30
540/540 - 3s - 5ms/step - accuracy: 0.9666 - loss: 0.1134 - val_accuracy: 0.9597 - val_loss: 0.1307
Epoch 5/30
540/540 - 3s - 5ms/step - accuracy: 0.9724 - loss: 0.0949 - val_accuracy: 0.9620 - val_loss: 0.1190
Epoch 6/30
540/540 - 3s - 5ms/step - accuracy: 0.9758 - loss: 0.0811 - val_accuracy: 0.9647 - val_loss: 0.1110
Epoch 7/30
540/540 - 3s - 6ms/step - accuracy: 0.9785 - loss: 0.0716 - val_accuracy: 0.9663 - val_loss: 0.1063
Epoch 8/30
540/540 - 3s - 5ms/step - accuracy: 0.9813 - loss: 0.0631 - val_accuracy: 0.9655 - val_loss: 0.1141
Epoch 9/30
540/540 - 3s - 5ms/step - accuracy: 0.9832 - loss: 0.0566 - val_accuracy: 0.9655 - val_loss: 0.1068

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step - accuracy: 0.9704 - loss: 0.0999
Test loss: 0.10. Test accuracy: 97.04%


In [ ]:
mg = cv2.imread('/content/cyfra.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = cv2.bitwise_not(img)
img = img / 255.0
img = img.reshape(1, 28, 28, 1)
prediction = model.predict(img)
predicted_label = np.argmax(prediction)
print(f'Predykcja modelu: {predicted_label}')
